# Prepare Data
load data from MySQL tables, include Bloomberg ID, request fileds and history date period

In [1]:
import mysql.connector
serverName = "awdiquant01.dev.hoopp.cloud"

cnx = mysql.connector.connect(user='quant', password='Password2017',
                              host=serverName,
                              database='cds_basis_trade')

In [2]:
def getRequestField(call_type):
    query = "SELECT field_name FROM `config.bps_request_field` WHERE call_type = '" + call_type + "'"
    cursor = cnx.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    
    return result

def getBloombergId():
    query = "SELECT bloomberg_id FROM `stream.reference_data` WHERE effective_date = CURRENT_DATE"
    cursor = cnx.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    
    return result

def getRequestOverride(call_type):
    query = "SELECT field_name, field_value FROM `config.bps_request_override` WHERE call_type = '" + call_type + "'"
    cursor = cnx.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    
    return result

#def saveBondHistoricData(bloombergid, pxlast, openprice):
def saveBondHistoricData(bond_historic_data):    
    
    query = ("insert into `history.bond_history_py` (`bloomberg_id`, `effective_date`, `px_last`, `open_price`) "
             "values (%(bloomberg_id)s, %(date)s, %(px_last)s, %(open)s)")
    cursor = cnx.cursor()
    for data in bond_historic_data:
        cursor.execute(query, data)
    
    cnx.commit()
    cursor.close()

    #print(query)

In [3]:
# embbed the call to main function
#request_bbgId = getBloombergId()
#request_field = getRequestField("BOND_HISTORIC")
#request_date = getRequestOverride("BOND_HISTORIC")

data_bond = [{
    'bloomberg_id' : 'abcdefg',
    'px_last' : 101.23,
    'open_price' : 876.322
    },
    {
    'bloomberg_id' : 'zxmyd',
    'px_last' : 202.23,
    'open_price' : 556.322
    },
]
#saveBondHistoricData(data_bond)


# Start BPIPE service
call BPIPE service by application authentication, and pass in the bloomberg Id with fields requested

In [4]:
import blpapi
from optparse import OptionParser
#from optparse import OptionParser
import time

In [5]:
TOKEN_SUCCESS = blpapi.Name("TokenGenerationSuccess")
TOKEN_FAILURE = blpapi.Name("TokenGenerationFailure")
AUTHORIZATION_SUCCESS = blpapi.Name("AuthorizationSuccess")
AUTHORIZATION_FAILURE = blpapi.Name("AuthorizationFailure")
RESPONSE_ERROR = blpapi.Name("responseError")

In [6]:
g_identity = None
g_identityCId = None


In [7]:
class SessionEventHandler(object):
    def getTimeStamp(self):
        return time.strftime("%Y/%m/%d %X")

    def processMiscEvents(self, event):
        timeStamp = self.getTimeStamp()
        for msg in event:
            print ("%s: %s" % (timeStamp, msg.messageType()))
            print (msg)

    def processEvent(self, event, session):
        try:
            return self.processMiscEvents(event)
        except blpapi.Exception as e:
            print ("Library Exception !!! %s" % e.description())
        return False


In [8]:
def parseCmdLine():
    parser = OptionParser(description="Retrieve reference data.")
    parser.add_option("-a",
                      "--ip",
                      dest="host",
                      help="server name or IP (default: %default)",
                      metavar="ipAddress",
                      default="localhost")
    parser.add_option("-p",
                      dest="port",
                      type="int",
                      help="server port (default: %default)",
                      metavar="tcpPort",
                      default=8194)

    (options, args) = parser.parse_args()

    return options


In [9]:
def authorize(session, cId):
    authQueue = blpapi.EventQueue()
    tokenReqId = blpapi.CorrelationId(99)
    session.generateToken(tokenReqId, authQueue)
    done = False
    while not done:
        event = authQueue.nextEvent(1000)
        if event.eventType() != blpapi.Event.TIMEOUT:
            if event.eventType() == blpapi.Event.TOKEN_STATUS:
                for msg in event:
                    if msg.messageType() == TOKEN_SUCCESS:
                        # Token generation success
                        print(msg)
                        if msg.hasElement("token"):
                            token = msg.getElementAsString("token")
                        done = True
                        break
                    elif msg.messageType() == TOKEN_FAILURE:
                        # Token generation failure
                        print(msg)
                        return
                    else:
                        print(msg)
                        break

    identity = session.createIdentity()
    authSvc = session.getService("//blp/apiauth")
    authRequest = authSvc.createAuthorizationRequest()
    authRequest.set("token", token)
    
    session.sendAuthorizationRequest(
        authRequest,
        identity,
        cId,
        authQueue)

    while True:
        # nextEvent() method below is called with a timeout to let
        # the program catch Ctrl-C between arrivals of new events
        event = authQueue.nextEvent(1000)
        if event.eventType() != blpapi.Event.TIMEOUT:
            for msg in event:
                if msg.messageType() == AUTHORIZATION_SUCCESS:
                    # Authorization successful
                    print(msg)
                    return identity
                elif msg.messageType() == AUTHORIZATION_FAILURE:
                    # Authorization failure
                    print(msg)
                    exit()
                else:
                    print(msg)
                    exit()
        else:
            print ("Timeout occurred during authorization")
            exit()


In [10]:
def main():
    host = "69.184.252.34"
    port = 8194
    auth = "APPLICATION"
    appName = "hoopp:Octopus1"
    dirName = ""
    
    #options = parseCmdLine()

    # Fill SessionOptions
    sessionOptions = blpapi.SessionOptions()
    #sessionOptions.setServerHost(options.host)
    #sessionOptions.setServerPort(options.port)
    sessionOptions.setServerHost(host)
    sessionOptions.setServerPort(port)

    # Authentication option
    if auth == "APPLICATION":
        # Set Application Authentication Option
        authOption = "AuthenticationMode=APPLICATION_ONLY;ApplicationAuthenticationType=APPNAME_AND_KEY;ApplicationName="
        authOption += appName
    elif auth == "USER_APP":
        # Set User and Application Authentication Option
        authOption = "AuthenticationMode=USER_AND_APPLICATION;"
        authOption += "AuthenticationType=OS_LOGON;"
        authOption += "ApplicationAuthenticationType=APPNAME_AND_KEY;"
        # ApplicationName is the entry in EMRS.
        authOption += "ApplicationName=" + appName
    elif auth == "DIRSVC":
        # Authenticate user using active directory service property
        authOption = "AuthenticationType=DIRECTORY_SERVICE;"
        authOption += "DirSvcPropertyName=" + dirName
    else:
        # Authenticate user using windows/unix login name
        authOption = "AuthenticationType=OS_LOGON"
    
    sessionOptions.setAuthenticationOptions(authOption)
    
    print ("authOptions = %s" % authOption)

    print ("Connecting to %s:%s" % (host, port))
    # Create a Session
    eventHandler = SessionEventHandler()
    session = blpapi.Session(sessionOptions, eventHandler.processEvent)

    # Start a Session
    if not session.start():
        print ("Failed to start session.")
        return

    try:
        # Open service to get historical data from
        if not session.openService("//blp/refdata"):
            print ("Failed to open //blp/refdata")
            return

        # Open authorization service
        if not session.openService("//blp/apiauth"):
            print ("Failed to open //blp/apiauth")
            return

        g_identityCId = blpapi.CorrelationId(1)
        g_identity = authorize(session, g_identityCId)
        
        if g_identity.getSeatType() == blpapi.Identity.INVALID_SEAT:
            print ("Authentication failed.")
            return
        elif g_identity.getSeatType() == blpapi.Identity.NONBPS:
            print ("Non-BPS seat type can not make historical data request.")
            return
         
        # Obtain previously opened service
        refDataService = session.getService("//blp/refdata")

        # get request data from database
        request_bbgId = getBloombergId()
        request_field = getRequestField("BOND_HISTORIC")
        request_date = getRequestOverride("BOND_HISTORIC")
        
        # Create historical request data
        request = refDataService.createRequest("HistoricalDataRequest")
        for row in request_bbgId:
            request.getElement("securities").appendValue(row[0] + "@MSG1 Corp")  
        for row in request_field:
            request.getElement("fields").appendValue(row[0])
        for row in request_date:
            if(row[0]=="STARTDATE"):
                request.set('startDate', row[1])
            if(row[0]=="ENDDATE"):
                request.set('endDate', row[1])
            
        #print(request)
        
        # Create and fill the request for the historical data
#         request = refDataService.createRequest("HistoricalDataRequest")
#         request.getElement("securities").appendValue("EI486050@MSG1 Corp")
#         request.getElement("securities").appendValue("EJ120557@MSG1 Corp")
#         request.getElement("fields").appendValue("PX_LAST")
#         request.getElement("fields").appendValue("OPEN")
#         request.set("startDate", "20180101")
#         request.set("endDate", "20181231")

        eventQueue = blpapi.EventQueue()
        cId = blpapi.CorrelationId(100)
        print ("Sending Request:", request)
        # Send the request
        session.sendRequest(request, g_identity, cId, eventQueue)

        # collect BPIPE result
        his_data = []
        
        # Process received events
        while(True):
            # We provide timeout to give the chance for Ctrl+C handling:
            ev = eventQueue.nextEvent(5000)
            if (ev.eventType() == blpapi.Event.PARTIAL_RESPONSE or
                ev.eventType() == blpapi.Event.RESPONSE):
                for msg in ev:
                    if msg.hasElement(RESPONSE_ERROR, True):
                        print ("Request error: %s" % (msg))
                        continue
                    
                    sec_data = {}
                    
                    #print msg
                    security_data = msg.getElement('securityData')
                    security_name = security_data.getElement('security')
                    field_data = security_data.getElement('fieldData')
                    
                    sec_data['bloomberg_id'] = security_name.getValueAsString().replace('@MSG1 Corp', '')
                    
                    #print (type(security_name), "Security: %s" % (security_name))
                    for i in range(field_data.numValues()):
                        # get row
                        fields = field_data.getValue(i)
                        
                        #if (fields.hasElement("date")):
                        #    field = fields.getElement("date")
                        #    print ("\t%s = %s" % (field.name(), field.getValue()))
                        
                        for j in range(fields.numElements()):
                            # process field data
                            field = fields.getElement(j)
                            #print ("\t\t%s = %s" % (field.name().getElementAsString(), field.getValue()))
                            sec_data[str(field.name()).lower()] = field.getValue()
                    
                        his_data.append(sec_data.copy())
                    
            elif (ev.eventType() == blpapi.Event.TIMEOUT):
                print ("Application timeout waiting for response. Canceling request.")
                session.cancel(cId)
                # print all other event messages
                for msg in ev:
                    print (msg)
            else:
                # print all other event messages
                for msg in ev:
                    print (msg)

            if ev.eventType() == blpapi.Event.RESPONSE:
                # Response completly received, so we could exit
                break
        
        print('historic data rows: ', len(his_data))
        saveBondHistoricData(his_data)
        
    finally:
        if g_identity.getSeatType() != blpapi.Identity.INVALID_SEAT: 
            session.cancel(g_identityCId)
        # Stop the session
        session.stop()

In [11]:
if __name__ == "__main__":
    print ("HistoryExample")
    try:

        main()
    except KeyboardInterrupt:
        print ("Ctrl+C pressed. Stopping...")


HistoryExample
authOptions = AuthenticationMode=APPLICATION_ONLY;ApplicationAuthenticationType=APPNAME_AND_KEY;ApplicationName=hoopp:Octopus1
Connecting to 69.184.252.34:8194
2018/01/08 23:10:37: SessionConnectionUp
SessionConnectionUp = {
    server = "69.184.252.34:8194"
    serverId = "16465-108716"
}

2018/01/08 23:10:37: SessionStarted
SessionStarted = {
    initialEndpoints[] = {
        initialEndpoints = {
            address = "69.184.252.34:8194"
        }
    }
}

2018/01/08 23:10:37: ServiceOpened
ServiceOpened = {
    serviceName = "//blp/refdata"
}

2018/01/08 23:10:37: ServiceOpened
ServiceOpened = {
    serviceName = "//blp/apiauth"
}

TokenGenerationSuccess = {
    token = "-v4;lgHKB5HI7d5fUvSm3RyTSHk7vgOo8OukqjQ1diRCv6NaUhYXKPftspRWPT/UBj4VyOq9uEfX/A2h
rqwM657oBroNpG3hYGh1EtUAyMQ1R+s=;Pfpx/jbALwiAfhb0dxHw1/SsiiAUGuyDPCQgt9fVNQZ38ACI0C2js0YP1u61CGCSdqcz2uH2ki+k
sy7l2WMwdWH/EiwjGvSYGCXZLWYaFy8=;"
}

AuthorizationSuccess = {
}

Sending Request: HistoricalDataRequest = {


historic data rows:  8366
2018/01/08 23:10:44: SessionConnectionDown
SessionConnectionDown = {
    server = "69.184.252.34:8194"
    serverId = "16465-108716"
}

2018/01/08 23:10:44: SessionTerminated
SessionTerminated = {
}

